# **Veri Seti Hazırlama Süreci**
Bu betik, veri setini eğitim için hazırlamak amacıyla aşağıdaki adımları gerçekleştirir:

1. **Veri Seti Dizini ve Sınıf Seçimi**
* Veri Seti Dizinleri:
Girdi, çıktı ve işlenmiş veri dizinlerini tanımlar.
* Sınıf Seçimi:
Projede kullanılacak hayvan sınıflarını belirler.

2. **Görüntü Sınırları ve Ön İşleme Ayarları**
* Sınıf Başına Görüntü Sınırı:
Her sınıf için maksimum 650 görüntü ile sınırlandırır.
* Görüntü Boyutlandırma:
Görüntüleri yeniden boyutlandırmak için hedef boyutu (örneğin, 128x128 piksel) ayarlar.

3. **Çıktı Dizinlerinin Kurulumu**
* Mevcut Dizinlerin Temizlenmesi:
Çıktı ve işlenmiş veri dizinlerini temizleyerek işlem öncesi temiz bir başlangıç sağlar.
* Yeni Dizinlerin Oluşturulması:
İşlenmiş görüntülerin kaydedilmesi için gerekli olan dizinleri yeniden oluşturur.

4. **Görüntü Kopyalama ve İşleme**
* Sınıflar Arasında Yineleme:
Belirtilen hayvan sınıfları arasında döngü yapar.
* Görüntülerin Çıktı Dizinine Kopyalanması:
Her sınıf için belirtilen sınıra kadar olan görüntüleri çıktı dizinine kopyalar.
* Görüntülerin İşlenmesi:
Görüntüleri yeniden boyutlandırır ve normalleştirerek işlenmiş dizine kaydeder.

In [2]:
# Gerekli kütüphaneleri içe aktar
import cv2
import numpy as np
import os
import shutil
from pathlib import Path

# Veri seti dizinleri (Dosya yapınıza göre ayarlayın)
data_dir = "/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2/JPEGImages"
output_dir = "/kaggle/working/selected_animals_dataset"
processed_dir = "/kaggle/working/processed_dataset"

# Seçilen hayvan sınıfları
selected_classes = [
    "collie", "dolphin", "elephant", "fox", "moose", 
    "rabbit", "sheep", "squirrel", "giant+panda", "polar+bear"
]

# Her sınıf için maksimum görüntü sayısı
max_images_per_class = 650

# İstenen görüntü boyutu
image_size = (128, 128)  # Model girdi boyutuna göre ayarlanabilir

# Çıktı ve işlenmiş dizinleri temizle ve yeniden oluştur
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

if os.path.exists(processed_dir):
    shutil.rmtree(processed_dir)
os.makedirs(processed_dir)

# Seçilen sınıfların görüntülerini kopyala ve işle
for animal_class in selected_classes:
    class_path = os.path.join(data_dir, animal_class)
    if os.path.exists(class_path):
        images = sorted(os.listdir(class_path))[:max_images_per_class]
        output_class_path = os.path.join(output_dir, animal_class)
        processed_class_path = os.path.join(processed_dir, animal_class)
        os.makedirs(output_class_path, exist_ok=True)
        os.makedirs(processed_class_path, exist_ok=True)

        for image in images:
            source_path = os.path.join(class_path, image)
            target_path = os.path.join(output_class_path, image)
            shutil.copy2(source_path, target_path)

            # Load image, resize and normalize
            img = cv2.imread(source_path)
            if img is not None:
                img_resized = cv2.resize(img, image_size)
                img_normalized = img_resized / 255.0
                processed_image_path = os.path.join(processed_class_path, image)
                cv2.imwrite(processed_image_path, (img_normalized * 255).astype(np.uint8))

print(f"Selected classes and the first {max_images_per_class} images have been copied to '{output_dir}', and processed images saved to '{processed_dir}'.")

Selected classes and the first 650 images have been copied to '/kaggle/working/selected_animals_dataset', and processed images saved to '/kaggle/working/processed_dataset'.


# **Eğitim ve Test Verilerinin Hazırlanması**
Bu betik, veri setini eğitim ve test için hazırlamak amacıyla aşağıdaki işlemleri gerçekleştirir:

1. **Veri Seti ve Etiketlerin Tanımlanması**
* İşlenmiş Veri Seti Dizini:
İşlenmiş veri seti dizinini tanımlar.
* Hayvan Sınıfları ve Etiketler:
Seçilen hayvan sınıflarını listeler ve bu sınıfları sayısal etiketlere dönüştürür.

2. **Veri Yükleme ve Normalizasyon**
* Görüntülerin İşlenmesi:
Her seçilen sınıf için işlenmiş görüntüler üzerinde iterasyon yapar.
* Piksel Değerlerinin Normalizasyonu:
Görüntülerin piksel değerlerini [0, 1] aralığına dönüştürür.

3. **Verilerin Dönüştürülmesi**
* NumPy Dönüşümü:
Görüntü ve etiket listelerini modelle uyumlu hale getirmek için NumPy dizilerine dönüştürür.

4. **Veri Setinin Bölünmesi**
* Eğitim ve Test Ayrımı:
Veri setini %70 eğitim ve %30 test olacak şekilde böler.
* Rastgelelik Kontrolü:
Sabit bir rastgelelik tohumu kullanarak bölünmenin reproducible (tekrarlanabilir) olmasını sağlar.


In [4]:
# Gerekli Kütüphanelerin İçe Aktarılması
import os
import numpy as np
from sklearn.model_selection import train_test_split

# İşlenmiş Veri Seti Dizini
processed_dir = "/kaggle/working/processed_dataset"

# Seçilen Hayvan Sınıfları
selected_classes = [
    "collie", "dolphin", "elephant", "fox", "moose", 
    "rabbit", "sheep", "squirrel", "giant+panda", "polar+bear"
]

# Veri ve Etiketlerin Hazırlanması
X = []  # Images
y = []  # Labels
class_mapping = {cls: idx for idx, cls in enumerate(selected_classes)}

# Görüntülerin Yüklenmesi ve Normalize Edilmesi
for animal_class in selected_classes:
    class_path = os.path.join(processed_dir, animal_class)
    if os.path.exists(class_path):
        images = sorted(os.listdir(class_path))

        for image in images:
            source_path = os.path.join(class_path, image)

            # Load image and normalize (already resized)
            img = cv2.imread(source_path)
            if img is not None:
                img_normalized = img / 255.0
                X.append(img_normalized)
                y.append(class_mapping[animal_class])

# Verilerin NumPy Dizilerine Dönüştürülmesi
X = np.array(X)
y = np.array(y)

# Veri Setinin Eğitim ve Test Olarak Ayrılması
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Bilgilendirme Çıktısı
print(f"Data has been split into training and testing sets.\nTrain size: {len(X_train)}, Test size: {len(X_test)}")


Data has been split into training and testing sets.
Train size: 4550, Test size: 1950


# **Eğitim Verileri İçin Veri Artırımı (Data Augmentation)**
1. **Veri Artırımı Başlatılması**
* ImageDataGenerator sınıfının bir örneği, aşağıdaki veri artırımı seçenekleriyle oluşturulur:
* **Döndürme (Rotation):** ±20 derece arasında rastgele döndürme.
* **Yatay ve Dikey Kaydırma:** Görüntüleri toplam genişliğin veya yüksekliğin %20'si kadar rastgele kaydırma.
* **Kesme Dönüşümleri (Shear Transformations):** Rastgele kesme dönüşümleri uygulanması.
* **Yakınlaştırma (Zoom):** Görüntülere %20 oranında rastgele yakınlaştırma.
* **Yatay Çevirme (Horizontal Flip):** Görüntüleri yatay eksende rastgele çevirme.
* **Doldurma Modu (Fill Mode):** Dönüşümler sonrası oluşan boş piksellerin nasıl doldurulacağını belirtir.

2. **Veri Artırımı Uygulanması**
* Eğitim görüntüleri (X_train) ve bunlara karşılık gelen etiketler (y_train), 32 görüntülük partiler halinde artırılır.

3. **Çıktı**
* Eğitim verilerine veri artırımı işleminin başarıyla uygulandığı doğrulanır.

In [5]:
# Gerekli Kütüphanelerin İçe Aktarılması
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Eğitim Verilerine Veri Artırımı Uygulanması
augmentation = ImageDataGenerator(
    rotation_range=20,  # Rastgele döndürme
    width_shift_range=0.2,  # Yatayda rastgele kaydırma
    height_shift_range=0.2,  # Dikeyde rastgele kaydırma
    shear_range=0.2,  # Kesme dönüşümleri
    zoom_range=0.2,  # Rastgele yakınlaştırma
    horizontal_flip=True,  # Rastgele yatay çevirme
    fill_mode='nearest'  # Boş pikseller için doldurma stratejisi
)


# Veri Artırımı İşleminin Uygulanması
augmented_data = augmentation.flow(X_train, y_train, batch_size=32)

# İşlem Çıktısı
print("Data augmentation has been applied to the training set.")

Data augmentation has been applied to the training set.


# **Görüntü Manipülasyonu ve Beyaz Dengesi Düzeltme**

Bu betik, görüntü veri setleri üzerinde iki temel işlemi gerçekleştirir:

1. **Görüntü Manipülasyonu** 
   - Parlaklık ve kontrast ayarlarını değiştirerek çeşitli ışıklandırma koşullarını simüle eder.
   - Manipüle edilmiş görüntüleri belirtilen çıktı dizinine kaydeder.

2. **Beyaz Dengesi Düzeltmesi Uygulama**
   - Gray World (Gri Dünya) varsayımını kullanarak görüntülerdeki renkleri normalize eder.
   - RGB kanallarının ortalama yoğunluklarına göre piksel değerlerini ayarlar.

## **Görüntü Manipülasyonu**

### **Genel Bakış**
`get_manipulated_images` fonksiyonu:
- Belirtilen giriş dizinindeki tüm görüntüler için parlaklık ve kontrast ayarlarını düzenler.
- Manipüle edilmiş görüntüleri belirtilen çıktı dizinine kaydeder.


### **Anahtar Parametreler**
- **Parlaklık Faktörü (Brightness Factor)**: Görüntülerin genel parlaklık seviyesini kontrol eder.
- **Kontrast Faktörü (Contrast Factor)**: Görüntülerdeki koyu ve açık alanlar arasındaki farkı ayarlar.


In [7]:
# Görüntü Manipülasyonu Fonksiyonu
def get_manipulated_images(input_dir, output_dir, brightness_factor=1.5, contrast_factor=1.2):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        output_class_path = os.path.join(output_dir, class_name)
        
        if not os.path.exists(output_class_path):
            os.makedirs(output_class_path)
        
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            
            if img is not None:
                # Parlaklık ve kontrast ayarlamalarını uygulayın
                manipulated_img = cv2.convertScaleAbs(img, alpha=contrast_factor, beta=brightness_factor * 50)
                
                # Düzenlenmiş resmi kaydet
                save_path = os.path.join(output_class_path, img_name)
                cv2.imwrite(save_path, manipulated_img)
    
    print(f"Manipulated images saved to {output_dir}")

# Dosya Yolları Tanımlamaları
input_dir = "/kaggle/working/processed_dataset"
manipulated_dir = "/kaggle/working/manipulated_images"

# İşlenmiş görseller üret
get_manipulated_images(input_dir, manipulated_dir)


# Beyaz Dengesi Düzeltme (Gray World Assumption) Fonksiyonu
def get_wb_images(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        output_class_path = os.path.join(output_dir, class_name)

        if not os.path.exists(output_class_path):
            os.makedirs(output_class_path)

        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)

            if img is not None:
                # Beyaz dengesi düzeltmesi için Gri Dünya Varsayımını uygulayın
                avg_b = np.mean(img[:, :, 0])
                avg_g = np.mean(img[:, :, 1])
                avg_r = np.mean(img[:, :, 2])
                avg_gray = (avg_b + avg_g + avg_r) / 3

                img[:, :, 0] = np.clip(img[:, :, 0] * (avg_gray / avg_b), 0, 255)
                img[:, :, 1] = np.clip(img[:, :, 1] * (avg_gray / avg_g), 0, 255)
                img[:, :, 2] = np.clip(img[:, :, 2] * (avg_gray / avg_r), 0, 255)

                # Düzeltilen görüntüyü kaydet
                save_path = os.path.join(output_class_path, img_name)
                cv2.imwrite(save_path, img.astype(np.uint8))

    print(f"White-balanced images saved to {output_dir}")

# Dosya Yolları Tanımlamaları
wb_output_dir = "/kaggle/working/wb_corrected_images"

# Gri Dünya Varsayımını Uygula
get_wb_images(manipulated_dir, wb_output_dir)

Manipulated images saved to /kaggle/working/manipulated_images
White-balanced images saved to /kaggle/working/wb_corrected_images


# **İşlenmiş ve Beyaz Dengeli Veri Kümelerinin Yüklenmesi**

Bu bölüm, işlenmiş ve beyaz dengeli veri kümelerinin eğitim veya değerlendirmede daha fazla kullanım için nasıl yükleneceğini gösterir.

In [10]:
# Yolların tanımlanması
manipulated_dir = "/kaggle/working/manipulated_images"  # İşlenmiş veri kümesi dizini
wb_output_dir = "/kaggle/working/wb_corrected_images"  # Beyaz dengeli veri kümesi dizini

# İşlenmiş veri kümesinin yüklenmesi
X_manipulated_train, y_manipulated_train = get_manipulated_images(manipulated_dir, manipulated_images)
print(f"Manipulated dataset loaded: {X_manipulated_train.shape}, {y_manipulated_train.shape}")

# Beyaz dengeli veri kümesinin yüklenmesi
X_wb_train, y_wb_train = get_wb_images(wb_output_dir, wb_corrected_images)
print(f"White-balanced dataset loaded: {X_wb_train.shape}, {y_wb_train.shape}")

Manipulated dataset loaded: (6500, 128, 128, 3), (6500,)
White-balanced dataset loaded: (6500, 128, 128, 3), (6500,)


# **Veri Kümelerini Birleştirme ve Modeli Eğitme**

Bu bölüm, veri kümelerini (orijinal, işlenmiş ve beyaz dengeli) birleştirme ve birleştirilmiş veriler üzerinde bir CNN modeli eğitme sürecini gösterir.

---

## **Veri Kümelerini Birleştirme**

### **Genel Bakış**
- Orijinal, işlenmiş ve beyaz dengeli veri kümeleri tek bir veri kümesinde birleştirilir.
- Modelin çeşitli veri dağıtımlarından öğrenmesini sağlar.

### **Kod**
```python
# Orijinal, İşlenmiş ve Beyaz Dengeli Veri Kümelerini Birleştirme
X_combined_train = np.concatenate([X_train, X_manipulated_train, X_wb_train], axis=0)
y_combined_train = np.concatenate([y_train, y_manipulated_train, y_wb_train], axis=0)


In [11]:
# Gerekli Kütüphanelerin İçe Aktarılması
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from sklearn.utils import shuffle

# Orijinal, İşlenmiş ve Beyaz Dengeli Veri Kümelerinin Birleştirilmesi
X_combined_train = np.concatenate([X_train, X_manipulated_train, X_wb_train], axis=0)
y_combined_train = np.concatenate([y_train, y_manipulated_train, y_wb_train], axis=0)

# Etiketlerin Bir Hot Codinge Dönüştürülmesi
y_combined_train_one_hot = to_categorical(y_combined_train, num_classes=10)
y_test_one_hot = to_categorical(y_test, num_classes=10)

# Birleştirilmiş Veri Setinin Karıştırılması
X_combined_train, y_combined_train_one_hot = shuffle(X_combined_train, y_combined_train_one_hot, random_state=42)

# CNN Modelin Tanımlanması
model = Sequential([
    Input(shape=(128, 128, 3)),
    Conv2D(32, (3, 3)),
    LeakyReLU(negative_slope=0.1),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3)),
    LeakyReLU(negative_slope=0.1),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3)),
    LeakyReLU(negative_slope=0.1),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(256, (3, 3)),
    LeakyReLU(negative_slope=0.1),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, kernel_regularizer=l2(0.02)),
    LeakyReLU(negative_slope=0.1),
    Dropout(0.7),
    Dense(10, activation='softmax')  # 10 classes
])

# Modelin Derlenmesi
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Modelin Eğitilmesi
history = model.fit(
    X_combined_train, y_combined_train_one_hot,
    epochs=20,  # Increased epochs
    batch_size=32,
    validation_data=(X_test, y_test_one_hot)
)

# Modelin Özeti
model.summary()

Epoch 1/20
549/549 ━━━━━━━━━━━━━━━━━━━━ 548s 991ms/step - accuracy: 0.2678 - loss: 7.2221 - val_accuracy: 0.4728 - val_loss: 5.4394
Epoch 2/20
549/549 ━━━━━━━━━━━━━━━━━━━━ 556s 981ms/step - accuracy: 0.4602 - loss: 5.2359 - val_accuracy: 0.6313 - val_loss: 4.0120
Epoch 3/20
549/549 ━━━━━━━━━━━━━━━━━━━━ 534s 972ms/step - accuracy: 0.5506 - loss: 4.0436 - val_accuracy: 0.6703 - val_loss: 3.1654
Epoch 4/20
549/549 ━━━━━━━━━━━━━━━━━━━━ 567s 981ms/step - accuracy: 0.6025 - loss: 3.1812 - val_accuracy: 0.7031 - val_loss: 2.5174
Epoch 5/20
549/549 ━━━━━━━━━━━━━━━━━━━━ 565s 986ms/step - accuracy: 0.6756 - loss: 2.4854 - val_accuracy: 0.7144 - val_loss: 2.1081
Epoch 6/20
549/549 ━━━━━━━━━━━━━━━━━━━━ 534s 973ms/step - accuracy: 0.7256 - loss: 1.9903 - val_accuracy: 0.8062 - val_loss: 1.5740
Epoch 7/20
549/549 ━━━━━━━━━━━━━━━━━━━━ 534s 972ms/step - accuracy: 0.7822 - loss: 1.6121 - val_accuracy: 0.8487 - val_loss: 1.3182
Epoch 8/20
549/549 ━━━━━━━━━━━━━━━━━━━━ 548s 998ms/step - accuracy: 0.8291 -

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 126, 126, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 126, 126, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 61, 61, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 61, 61, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 28, 28, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 12, 12, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 12, 12, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 12, 12, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 9216)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       1,179,776 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 4,712,288 (17.98 MB)

 Trainable params: 1,570,442 (5.99 MB)

 Non-trainable params: 960 (3.75 KB)

 Optimizer params: 3,140,886 (11.98 MB)

# **Modelin İşlenmiş Test Setinde Değerlendirilmesi**

## **Genel Bakış**
Bu bölüm, parlaklık ve kontrast ayarlamaları içeren görüntüler içeren işlenmiş test setindeki modelin performansını değerlendirir. Amaç, modelin değiştirilmiş ışık koşullarına ne kadar iyi genelleştirildiğini değerlendirmektir.

## **Adımlar**

### **1. İşlenmiş Test Setini Yükle**
- İşlenmiş veri seti belirtilen dizinden yüklenir.
- Görüntüler ve etiketler çıkarılır.

#### **Kod**
```python
manipulated_dir = "/kaggle/working/manipulated_images" # Gerektiği gibi yolu ayarlayın

# İşlenmiş test görüntülerini ve etiketlerini yükleyin
X_manipulated_test, y_manipulated_test = load_images_from_directory(manipulated_dir, selected_classes)
print(f"Manipulated Test Set Loaded: {X_manipulated_test.shape}, {y_manipulated_test.shape}")

In [13]:
# Yüklenen manipüle edilmiş test seti
manipulated_dir = "/kaggle/working/manipulated_images"  # Gerektiği gibi yolu ayarlayın

# İşlenmiş test görüntülerinin ve etiketlerinin yüklenmesi
X_manipulated_test, y_manipulated_test = load_images_from_directory(manipulated_dir, selected_classes)

print(f"Manipulated Test Set Loaded: {X_manipulated_test.shape}, {y_manipulated_test.shape}")

# İşlenmiş test etiketlerini tek sıcak kodlamaya dönüştür
y_manipulated_test_one_hot = to_categorical(y_manipulated_test, num_classes=10)

# Modeli manipüle edilmiş test kümesinde değerlendirin
manipulated_loss, manipulated_accuracy = model.evaluate(X_manipulated_test, y_manipulated_test_one_hot, verbose=2)
print(f"Manipulated Test Loss: {manipulated_loss}, Manipulated Test Accuracy: {manipulated_accuracy}")

Manipulated Test Set Loaded: (6500, 128, 128, 3), (6500,)
204/204 - 40s - 198ms/step - accuracy: 0.9895 - loss: 0.3466
Manipulated Test Loss: 0.3465944528579712, Manipulated Test Accuracy: 0.9895384907722473


# **Beyaz Dengeli Test Setinde Modelin Değerlendirilmesi**

## **Genel Bakış**
Bu bölüm, renk normalizasyonu için Gri Dünya varsayımı kullanılarak işlenen beyaz dengeli test setindeki modelin performansını değerlendirir Modelin amacı beyaz dengesi düzeltmesinin sınıflandırma doğruluğunu iyileştirip iyileştirmediğini belirlemektir.

## **Adımlar**

### **1. Beyaz Dengeli Test Setini Yükle**
- Beyaz dengeli veri seti belirtilen dizinden yüklenir.
- Görüntüler ve etiketler çıkarılır.

#### **Kod**
```python
wb_output_dir = "/kaggle/working/wb_corrected_images" # Yolu gerektiği gibi ayarlayın

# Beyaz dengeli test görüntüleri ve etiketleri yükleyin
X_wb_test, y_wb_test = load_images_from_directory(wb_output_dir, selected_classes)
print(f"White-Balanced Test Set Loaded: {X_wb_test.shape}, {y_wb_test.shape}")


In [15]:
# Beyaz dengesi sağlanmış test setinin yüklenmesi
wb_output_dir = "/kaggle/working/wb_corrected_images"  # Gerektiği gibi yolu ayarlayın

# Beyaz dengesi sağlanmış test görüntülerin ve etiketlerin yüklenmesi
X_wb_test, y_wb_test = load_images_from_directory(wb_output_dir, selected_classes)

print(f"White-Balanced Test Set Loaded: {X_wb_test.shape}, {y_wb_test.shape}")

y_wb_test_one_hot = to_categorical(y_wb_test, num_classes=10)

wb_loss, wb_accuracy = model.evaluate(X_wb_test, y_wb_test_one_hot, verbose=2)
print(f"White-Balanced Test Loss: {wb_loss}, White-Balanced Test Accuracy: {wb_accuracy}")

White-Balanced Test Set Loaded: (6500, 128, 128, 3), (6500,)
204/204 - 40s - 195ms/step - accuracy: 0.9966 - loss: 0.3308
White-Balanced Test Loss: 0.33081671595573425, White-Balanced Test Accuracy: 0.9966154098510742


# **Modelin Orijinal Test Setinde Değerlendirilmesi**

## **Genel Bakış**
Bu adımda, bir temel performans belirlemek için modelin orijinal (değiştirilmemiş) test veri setindeki performansını değerlendiriyoruz.

## **Adımlar**

### **1. Etiketleri Tek Sıcak Kodlamaya Dönüştür**
- Etiketler, kategorik sınıflandırma modeliyle uyumluluk için tek sıcak kodlanmış bir biçime dönüştürülür.

#### **Kod**
```python
# Orijinal test etiketlerini tek sıcak kodlamaya dönüştür

y_test_one_hot = to_categorical(y_test, num_classes=10)


In [16]:
# Orijinal test etiketlerini tek hot codinge dönüştür
y_test_one_hot = to_categorical(y_test, num_classes=10)

# Modeli orijinal test setinde değerlendirin
original_loss, original_accuracy = model.evaluate(X_test, y_test_one_hot, verbose=2)

# Sonuçların Çıktısı
print(f"Original Test Loss: {original_loss}, Original Test Accuracy: {original_accuracy}")

61/61 - 12s - 195ms/step - accuracy: 0.9492 - loss: 0.4589
Original Test Loss: 0.4589402675628662, Original Test Accuracy: 0.9492307901382446
